In [10]:
from ovito.io import *
from ovito.modifiers import *
from ovito.data import *
from ovito.data import CutoffNeighborFinder

import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [11]:
def datacollection2DataFrame(data):
    pos_prop = data.particles["Position"][:]
    type_prop = data.particles["Particle Type"][:]
    id_prop = data.particles["Particle Identifier"][:]
    combined_data = np.column_stack((id_prop, type_prop, pos_prop))
    df = pd.DataFrame(combined_data, columns=['id', 'type', 'x', 'y', 'z'])
    return df

In [12]:
def cal_dist(pos1, pos2):
    dist = np.sqrt(sum((pos1 - pos2) * (pos1 - pos2)))
    return dist

In [13]:
def cal_neigh_dist(id_of_center_atom, data, index_of_neigh):
    pos_center_atom = data.particles["Position"][id_of_center_atom]
    neigh_id = index_of_neigh[id_of_center_atom]
    distance = []
    for index in neigh_id:
        pos_neigh = data.particles["Position"][index]
        distance.append(cal_dist(pos_center_atom, pos_neigh))
    return distance

In [14]:
def cutoff_bypairtype(type1, type2):
    a = 2
    cutoff_11 = 0.00022 * a
    cutoff_12 = 0.0003 * a
    cutoff_22 = 0.00025 * a

    checknumber = ((type1 + type2) % 3)
    if checknumber == 2:
        return cutoff_11
    elif checknumber == 0:
        return cutoff_12
    else:
        return cutoff_22

In [15]:
def findCrackedIDAndBondCenter(data, dist_list, index_of_neigh, data_df):
    cracked_id = []
    broken_bond_centers = []
    for index in range(data.particles.count):
        for len_idx, dist_value in enumerate(dist_list[index]):
            neigh_index = index_of_neigh[index][len_idx]
            center_type = data_df['type'][index]
            neigh_type = data_df['type'][neigh_index]
            if dist_value > cutoff_bypairtype(center_type, neigh_type):
                cracked_id.append(neigh_index)
                pos1 = data.particles["Position"][index]
                pos2 = data.particles["Position"][neigh_index]
                center_pos = (pos1 + pos2) / 2
                broken_bond_centers.append(center_pos)
    broken_bond_centers = np.array(broken_bond_centers)
    return cracked_id, broken_bond_centers

In [ ]:
def process_file(file_path):
    pipeline = import_file(file_path, sort_particles=True)
    
    data = pipeline.compute(0)
    numOfFinalFrame = pipeline.source.num_frames - 1
    data_final = pipeline.compute(numOfFinalFrame)

    data_df = datacollection2DataFrame(data)
    data_final_df = datacollection2DataFrame(data_final)

    index_of_neigh = [[] for _ in range(data.particles.count)]
    cutoff = 0.0002001
    finder = CutoffNeighborFinder(cutoff, data)
    
    for index in range(data.particles.count):
        temp = []
        for neigh in finder.find(index):
            temp.append(neigh.index)
        index_of_neigh[index] = temp

    dist_list = [[] for _ in range(data_final.particles.count)]
    for index in range(data_final.particles.count):
        dist_list[index] = cal_neigh_dist(index, data_final, index_of_neigh)

    cracked_id, broken_bond_centers = findCrackedIDAndBondCenter(data_final, dist_list, index_of_neigh, data_df)


    pos_final = data_final.particles["Position"]

    crack_pos_final = pos_final[cracked_id][:, :2]

    file_name = os.path.splitext(os.path.basename(file_path))[0]
    os.makedirs("crack_png", exist_ok=True)

    crack_pos_first = data.particles["Position"][cracked_id][:, :2]

    fig, ax = plt.subplots(1, 2, figsize=(60, 30))
    
    ax[0].scatter(data_df['x'], data_df['y'], s=10, c=data_df['type'], cmap='viridis')
    ax[0].scatter(crack_pos_first[:, 0], crack_pos_first[:, 1], s=25, c='red')

    ax[1].scatter(data_final_df['x'], data_final_df['y'], s=10, c=data_final_df['type'], cmap='viridis')
    ax[1].scatter(crack_pos_final[:, 0], crack_pos_final[:, 1], s=25, c='red')

    plt.savefig(f"crack_png/{file_name}.png", dpi=300)
    plt.close()

In [17]:
input_directory = "data"
file_list = [os.path.join(input_directory, file) for file in os.listdir(input_directory) if file.endswith(".dump")]


In [18]:

for file_path in file_list:
    print(f"Processing file: {file_path}")
    process_file(file_path)

print("All files processed.")

Processing file: data/ani_density_[50]location[normal]_fibersize[normal]_seed[1][21].dump


KeyboardInterrupt: Operation has been canceled by the user.